In [211]:
%store -r train_mod 
%store -r test_mod

# Импорт

In [212]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
import re
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder

In [213]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)

Python       : 3.10.4 (tags/v3.10.4:9d38120, Mar 23 2022, 23:13:41) [MSC v.1929 64 bit (AMD64)]
Numpy        : 1.22.3


# Функции

In [214]:
#Словарь соответствий брендов странас происхождения
countries_dict = {'Germany': ['AUDI','BMW','OPEL','PORSCHE','VOLKSWAGEN','MERCEDES'],
                  'Japan':   ['HONDA','INFINITI','LEXUS','MAZDA','MITSUBISHI','NISSAN','SUBARU','SUZUKI','TOYOTA'],
                  'USA':     ['CADILLAC','CHEVROLET','CHRYSLER','DODGE','FORD','JEEP'],
                  'China':   ['CHERY','GEELY','GREAT_WALL'],
                  'UK':      ['JAGUAR','MINI','LAND_ROVER'],
                  'Korea':   ['DAEWOO','HYUNDAI','KIA','SSANG_YONG'],
                  'Sweden':  ['VOLVO'],
                  'Czech':   ['SKODA']}

In [215]:
#Функция для заполнения столбца со страной происхождения бренда
def get_country(x):
    if x in countries_dict['Germany']:
        x = 'Germany'
    elif x in countries_dict['Japan']:
        x = 'Japan'
    elif x in countries_dict['USA']:
        x = 'USA'
    elif x in countries_dict['China']:
        x = 'China'
    elif x in countries_dict['UK']:
        x = 'UK'
    elif x in countries_dict['Korea']:
        x = 'Korea'
    elif x in countries_dict['Sweden']:
        x = 'Sweden'
    else:
        x = 'Czech'
    return x
#Наверное, эта функция должна выглядеть не так колхозно


#Функция для просмотра и сравнения типов данных в столбцах
def type_comparison(column):
    print('TEST')
    display(test_mod[column].sample(5))
    print('TRAIN')
    display(train_mod[column].sample(5))  
    
    
    
#Функция для приведения значений столбца bodyType к единому формату
def get_bodyType(x):
    row = str(x).lower()
    bodyType = re.findall('[а-яё]+', row)
    if bodyType == []:
        return None
    return str(bodyType[0])



#Функция для приведения значений столбца engineDisplacement к единому формату
def get_EngineDisplacement(x):
    row = str(x)
    engine = re.findall('\d\.\d', row)
    if engine == []:
        return None
    return float(engine[0])



#Функция для извлечения значения объёма двигателя из столбца name
def pull_EngineDisplacement(x):
    row = str(x)
    engine = re.findall('\d\.\d', row)
    if len(engine) !=0:
        return float(engine[0])
    else:
        return 0

    
    
#Список имён столбцов под удаление. Будет заполняться по ходу первичного анализа.
columns_to_drop = []

In [216]:
train2 = train_mod.copy()
test2 = test_mod.copy()

##### Определение общих и различающихся столбцов в TRAIN и TEST

In [217]:
#Общие столбцы
common_columns = set(test_mod.columns).intersection(train_mod.columns)

#Только в TEST
test_only_columns = set(test_mod.columns).difference(train_mod.columns)

#Только в TRAIN
train_only_columns = set(train_mod.columns).difference(test_mod.columns)

Как можно видеть, формат данных в части столбцов с одинаковыми названиями не совпадает. Также необходимо исследовать содержаимое ячеек.

##### Бренды в TRAIN и TEST

In [218]:
display(train_mod.brand.unique())
print('Всего в TRAIN',len(train_mod.brand.unique()),'брендов.',sep=' ')

array(['AUDI', 'BMW', 'CADILLAC', 'CHERY', 'CHEVROLET', 'CHRYSLER',
       'CITROEN', 'DAEWOO', 'DODGE', 'FORD', 'GEELY', 'HONDA', 'HYUNDAI',
       'INFINITI', 'JAGUAR', 'JEEP', 'KIA', 'LEXUS', 'MAZDA', 'MINI',
       'MITSUBISHI', 'NISSAN', 'OPEL', 'PEUGEOT', 'PORSCHE', 'RENAULT',
       'SKODA', 'SUBARU', 'SUZUKI', 'TOYOTA', 'VOLKSWAGEN', 'VOLVO',
       'GREAT_WALL', 'LAND_ROVER', 'MERCEDES', 'SSANG_YONG'], dtype=object)

Всего в TRAIN 36 брендов.


In [219]:
display(test_mod.brand.unique())
print('Всего в TEST',len(test_mod.brand.unique()),'брендов.',sep=' ')

array(['SKODA', 'AUDI', 'HONDA', 'VOLVO', 'BMW', 'NISSAN', 'INFINITI',
       'MERCEDES', 'TOYOTA', 'LEXUS', 'VOLKSWAGEN', 'MITSUBISHI'],
      dtype=object)

Всего в TEST 12 брендов.


Имеет смысл создать столбец со странами происхождения брендов.

In [220]:
test_mod['Country_of_origin'] = test_mod['brand'].apply(get_country)
train_mod['Country_of_origin'] = train_mod['brand'].apply(get_country)

## bodyType

In [221]:
column = 'bodyType'
type_comparison(column)

TEST


198                лифтбек
2493     внедорожник 5 дв.
12840                седан
29057                седан
34318                седан
Name: bodyType, dtype: object

TRAIN


6807                 Седан
28232                Седан
24925        Хэтчбек 5 дв.
19147    Внедорожник 5 дв.
51087    Внедорожник 5 дв.
Name: bodyType, dtype: object

Необходимо почистить эти данные и привести к единому формату.

In [222]:
test_mod['bodyType'] = test_mod.bodyType.apply(get_bodyType)
train_mod['bodyType'] = train_mod.bodyType.apply(get_bodyType)

In [223]:
column = 'bodyType'
type_comparison(column)

TEST


9968           седан
31573          седан
12150    внедорожник
29202    внедорожник
9754           седан
Name: bodyType, dtype: object

TRAIN


38040    внедорожник
48678    внедорожник
37367          седан
79194    внедорожник
7652            купе
Name: bodyType, dtype: object

## color

In [224]:
column = 'color'
type_comparison(column)

TEST


27501    серебристый
841            белый
10273         чёрный
3157     серебристый
4680          чёрный
Name: color, dtype: object

TRAIN


13070    97948F
67581    040001
12785    FAFBFB
25848    FAFBFB
74158    FAFBFB
Name: color, dtype: object

Здесь также требуется приведение к единому формату

In [225]:
train_mod.color.unique()

array(['040001', 'EE1D19', '0000CC', 'CACECB', '007F00', 'FAFBFB',
       '97948F', '22A0F8', '660099', '200204', 'C49648', 'DEA522',
       '4A2197', 'FFD600', 'FF8649', 'FFC0CB'], dtype=object)

In [226]:
test_mod.color.unique()

array(['синий', 'чёрный', 'серый', 'коричневый', 'белый', 'пурпурный',
       'бежевый', 'серебристый', 'красный', 'зелёный', 'жёлтый',
       'голубой', 'оранжевый', 'фиолетовый', 'золотистый', 'розовый'],
      dtype=object)

In [227]:
#Преобразование
train_mod = train_mod.replace({'color':
                            {'040001': 'чёрный',
                            'EE1D19': 'красный',
                            '0000CC': 'синий',
                            'CACECB': 'серебристый',
                            '007F00': 'зелёный',
                            'FAFBFB': 'белый',
                            '97948F': 'серый',
                            '22A0F8': 'голубой',
                            '660099': 'фиолетовый',
                            '200204': 'коричневый',
                            'C49648': 'бежевый',
                            'DEA522': 'золотистый',
                            '4A2197': 'пурпурный',
                            'FFD600': 'жёлтый',
                            'FF8649': 'оранжевый',
                            'FFC0CB': 'розовый'}})

In [228]:
column = 'color'
type_comparison(column)

TEST


17518     чёрный
32257    бежевый
29841      синий
1388       серый
34008      белый
Name: color, dtype: object

TRAIN


59312    зелёный
23283      синий
7243       серый
59271     чёрный
8383      чёрный
Name: color, dtype: object

## description

In [229]:
column = 'description'
type_comparison(column)

TEST


32239    Продаю Лансер- Рулевая рейка и Гур после ремон...
3884     Продаю своего немца! Автомобиль в хорошем техн...
14338    Продаю свой Личный автомобиль , так как требую...
16954    Комфорт: 10. Безопасность: 6. Салон: 8. Обзор:...
7021     Авто в хорошем состояние. Езжу каждый день. Дв...
Name: description, dtype: object

TRAIN


49366    АВТОМИР - крупнейший дилер на российском автор...
64771    Автомобиль в хорошем состоянии все работает ис...
49837    Машинка в идеальном состояние!!!!!делать вообщ...
28987    Пройдены  ТО у дилера. Сервисная книжка. \nОкл...
39554    машина по технической части в полном порядке, ...
Name: description, dtype: object

Пока неясно можно ли как-либо использовать этот столбец

## engineDisplacement

In [230]:
column = 'engineDisplacement'
type_comparison(column)

TEST


21581    3.5 LTR
848      1.6 LTR
21452    2.5 LTR
2780     3.0 LTR
30258    1.6 LTR
Name: engineDisplacement, dtype: object

TRAIN


32432    3.0d
31298    FX35
37151    2.5d
66614     2.4
7257     520d
Name: engineDisplacement, dtype: object

Необходимо привести данные к единому формату

In [231]:
train_mod['engineDisplacement'] = train_mod.engineDisplacement.apply(get_EngineDisplacement)
test_mod['engineDisplacement'] = test_mod.engineDisplacement.apply(get_EngineDisplacement)

In [232]:
column = 'engineDisplacement'
type_comparison(column)

TEST


11039    2.0
1652     1.4
454      1.4
16771    2.8
7497     4.4
Name: engineDisplacement, dtype: float64

TRAIN


8873     NaN
54878    1.8
30042    NaN
507      1.8
83610    NaN
Name: engineDisplacement, dtype: float64

После преобразования в столбцах появились пропуски

In [233]:
test_mod[test_mod['engineDisplacement'].isna()]

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,...,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,Country_of_origin
5241,внедорожник,AUDI,https://auto.ru/cars/used/sale/audi/e_tron/110...,белый,NaN,Audi e-tron Prestige quattro от Moscow Tesla C...,NaN,408 N12,"{""asr"":true,""esp"":true,""usb"":true,""e-adjustmen...",электро,...,автоматическая,EUROPEAN,1 владелец,1 год и 3 месяца,Оригинал,полный,Левый,Не требует ремонта,Растаможен,Germany
7992,хэтчбек,BMW,https://auto.ru/cars/used/sale/bmw/i3/11010431...,синий,NaN,"Атомобиль из Германии,без пробега по РФ.\nНа г...",NaN,170 N12,NaN,электро,...,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,задний,Левый,Не требует ремонта,Растаможен,Germany
8252,хэтчбек,BMW,https://auto.ru/cars/used/sale/bmw/i3/11012780...,серый,NaN,"Пробег 54000 км, запас хода на зарядке 130 км+...",NaN,170 N12,"{""park-assist-f"":true,""park-assist-r"":true}",электро,...,автоматическая,EUROPEAN,1 владелец,1 месяц,Оригинал,задний,Левый,Не требует ремонта,Растаможен,Germany
8350,хэтчбек,BMW,https://auto.ru/cars/used/sale/bmw/i3/11002716...,белый,"{""id"":""21624597"",""name"":""i3 120 Ah"",""available...","Автомобиль в наличии Москве , с ПТС , производ...",NaN,170 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",электро,...,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,задний,Левый,Не требует ремонта,Растаможен,Germany
8684,хэтчбек,BMW,https://auto.ru/cars/used/sale/bmw/i3/11011745...,чёрный,NaN,BMW I 3 REX\nфевраль 2017 года выпуска.\nПриоб...,NaN,170 N12,"{""cruise-control"":true,""glonass"":true,""navigat...",электро,...,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,задний,Левый,Не требует ремонта,Растаможен,Germany
8722,хэтчбек,BMW,https://auto.ru/cars/used/sale/bmw/i3/11000973...,голубой,NaN,\nПрибыло из Америки в августе 2020 года. Полн...,NaN,170 N12,NaN,электро,...,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,задний,Левый,Не требует ремонта,Растаможен,Germany
8917,хэтчбек,BMW,https://auto.ru/cars/used/sale/bmw/i3/11008715...,бежевый,NaN,-Состояние нового автомобиля .\n-Carfax вся ис...,NaN,170 N12,"{""glonass"":true,""roof-rails"":true}",электро,...,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,задний,Левый,Не требует ремонта,Растаможен,Germany
10817,хэтчбек,BMW,https://auto.ru/cars/used/sale/bmw/i3/11012664...,серый,NaN,Продаю BMW I3 +REX .\n\nПолноценный электромоб...,NaN,170 N12,NaN,электро,...,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,задний,Левый,Не требует ремонта,Растаможен,Germany
10936,хэтчбек,BMW,https://auto.ru/cars/used/sale/bmw/i3/10996733...,чёрный,NaN,Автомобиль BMW i3 (полностью электрический )за...,NaN,170 N12,"{""cruise-control"":true,""asr"":true,""tinted-glas...",электро,...,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,задний,Левый,Не требует ремонта,Растаможен,Germany
11043,хэтчбек,BMW,https://auto.ru/cars/used/sale/bmw/i3/10993761...,серый,NaN,Состояние нового автомобиля . \nФото ИМЕННО ЭТ...,NaN,170 N12,"{""glonass"":true,""airbag-driver"":true,""aux"":tru...",электро,...,автоматическая,EUROPEAN,1 владелец,8 месяцев,Оригинал,задний,Левый,Не требует ремонта,Растаможен,Germany


In [234]:
train_mod[train_mod['engineDisplacement'].isna()]

,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,...,Состояние,Владельцы,ПТС,Таможня,Владение,price,start_date,hidden,model,Country_of_origin
45,седан,AUDI,чёрный,бензин,1990.0,S4 2.2 AT (230 л.с.) 4WD,4.0,1992,SEDAN AUTOMATIC S4,AUTOMATIC,...,NaN,3.0,DUPLICATE,True,NaN,220000.0,2020-07-09T06:57:21Z,NaN,100,Germany
345,седан,AUDI,чёрный,бензин,2012.0,ultra 1.4 AMT (150 л.с.),4.0,2016,SEDAN ROBOT ultra,ROBOT,...,NaN,2.0,ORIGINAL,True,NaN,1254090.0,2020-09-05T16:01:05Z,NaN,A3,Germany
383,седан,AUDI,белый,бензин,2012.0,ultra 1.4 AMT (150 л.с.),4.0,2015,SEDAN ROBOT ultra,ROBOT,...,NaN,1.0,ORIGINAL,True,NaN,870000.0,2020-07-28T11:55:47Z,NaN,A3,Germany
658,универсал,AUDI,чёрный,дизель,2004.0,8V 2.0d MT (140 л.с.),5.0,2005,WAGON_5_DOORS MECHANICAL 8V,MECHANICAL,...,NaN,3.0,ORIGINAL,True,"{'year': 2020, 'month': 2}",380000.0,2020-09-08T19:06:10Z,NaN,A4,Germany
671,универсал,AUDI,чёрный,дизель,2004.0,8V 2.0d CVT (140 л.с.),5.0,2007,WAGON_5_DOORS VARIATOR 8V,VARIATOR,...,NaN,3.0,ORIGINAL,True,NaN,395000.0,2020-07-26T20:05:31Z,NaN,A4,Germany
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89238,внедорожник,SSANG_YONG,серебристый,дизель,2007.0,6-speed 2.0d AT (141 л.с.) 4WD,5.0,2010,ALLROAD_5_DOORS AUTOMATIC 6-speed,AUTOMATIC,...,NaN,3.0,ORIGINAL,True,"{'year': 2018, 'month': 3}",455000.0,2020-05-26T14:21:34Z,NaN,KYRON,Korea
89239,внедорожник,SSANG_YONG,чёрный,бензин,2007.0,6-speed 2.3 AT (150 л.с.) 4WD,5.0,2009,ALLROAD_5_DOORS AUTOMATIC 6-speed,AUTOMATIC,...,NaN,2.0,ORIGINAL,True,NaN,525000.0,2020-05-31T06:33:06Z,NaN,KYRON,Korea
89242,внедорожник,SSANG_YONG,серый,дизель,2007.0,6-speed 2.0d AT (141 л.с.) 4WD,5.0,2012,ALLROAD_5_DOORS AUTOMATIC 6-speed,AUTOMATIC,...,NaN,2.0,ORIGINAL,True,NaN,650000.0,2020-02-26T13:40:39Z,NaN,KYRON,Korea
89247,пикап,SSANG_YONG,серебристый,дизель,1998.0,Sports 2.9d MT (120 л.с.) 4WD,4.0,2006,PICKUP_TWO MECHANICAL Sports,MECHANICAL,...,NaN,1.0,ORIGINAL,True,NaN,385000.0,2020-08-30T12:58:53Z,NaN,MUSSO,Korea


Возможно, эти пропуски мы сможем заполнить, взяв информацию из столбца 'name'

In [235]:
test_mod[test_mod['engineDisplacement'].isna()]['name']

5241     55 Electro AT (301 кВт) 4WD
7992            Electro AT (126 кВт)
8252            Electro AT (126 кВт)
8350            Electro AT (126 кВт)
8684       94Ah Electro AT (126 кВт)
8722            Electro AT (126 кВт)
8917       94Ah Electro AT (126 кВт)
10817           Electro AT (126 кВт)
10936           Electro AT (126 кВт)
11043      94Ah Electro AT (126 кВт)
11302            Electro AT (81 кВт)
11682           Electro AT (111 кВт)
11714           Electro AT (111 кВт)
11719            Electro AT (81 кВт)
11754            Electro AT (81 кВт)
11878            Electro AT (81 кВт)
11974            Electro AT (81 кВт)
12072            Electro AT (81 кВт)
12085            Electro AT (81 кВт)
12100      30kWh Electro AT (81 кВт)
12250            Electro AT (81 кВт)
12287            Electro AT (81 кВт)
12334            Electro AT (81 кВт)
12466      30kWh Electro AT (81 кВт)
12481            Electro AT (81 кВт)
12558            Electro AT (81 кВт)
12582      30kWh Electro AT (81 кВт)
1

Очевидно что в TEST объём двигателя отсутствует исключительно у электромобилей. Что вполне логично. Стоит заполнить пропуски нолями

In [236]:
test_mod['engineDisplacement'].fillna(0, inplace = True)

In [237]:
train_mod[train_mod['engineDisplacement'].isna()]['name']

45             S4 2.2 AT (230 л.с.) 4WD
345            ultra 1.4 AMT (150 л.с.)
383            ultra 1.4 AMT (150 л.с.)
658               8V 2.0d MT (140 л.с.)
671              8V 2.0d CVT (140 л.с.)
                      ...              
89238    6-speed 2.0d AT (141 л.с.) 4WD
89239     6-speed 2.3 AT (150 л.с.) 4WD
89242    6-speed 2.0d AT (141 л.с.) 4WD
89247     Sports 2.9d MT (120 л.с.) 4WD
89252     Sports 2.9d AT (120 л.с.) 4WD
Name: name, Length: 20387, dtype: object

В TRAIN ситуация совсем иная. С этим ещё предстоит разобраться.

## enginePower

In [238]:
column = 'enginePower'
type_comparison(column)

TEST


16544    122 N12
29302    306 N12
19223    146 N12
388      152 N12
2790     190 N12
Name: enginePower, dtype: object

TRAIN


65657    167.0
42393     75.0
53756     70.0
48895    144.0
1591     220.0
Name: enginePower, dtype: float64

Необходимо привести данные к единому формату

In [239]:
#Преобразование
train_mod.enginePower = train_mod.enginePower.convert_dtypes(int)
test_mod["enginePower"] = test_mod["enginePower"].apply(lambda x : (int)(x.split(' ')[0]))

In [240]:
column = 'enginePower'
type_comparison(column)

TEST


10100    150
9295     193
31652    272
15370    280
16255    612
Name: enginePower, dtype: int64

TRAIN


39498    105
83880    585
10950    245
20794    125
50080    182
Name: enginePower, dtype: Int64

Проверим не образовались ли пропуски в этом столбце

In [241]:
test_mod[test_mod['enginePower'].isna()]

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,...,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,Country_of_origin


In [242]:
train_mod[train_mod['enginePower'].isna()]

,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,...,Состояние,Владельцы,ПТС,Таможня,Владение,price,start_date,hidden,model,Country_of_origin
24624,None,HONDA,серебристый,универсал,NaN,NaN,NaN,1989,NaN,NaN,...,NaN,3.0,ORIGINAL,True,NaN,120000.0,2020-04-10T16:07:57Z,NaN,CIVIC,Japan


Образовался один пропуск. Строка с этим пропуском в целом содержит мало данных и должна быть удалена

In [243]:
train_mod.drop(24624, inplace = True)

## fuelType

In [244]:
column = 'fuelType'
type_comparison(column)

TEST


10382    бензин
28277    бензин
25664    бензин
3973     дизель
6102     бензин
Name: fuelType, dtype: object

TRAIN


15868    бензин
85310    дизель
29252    бензин
88234    дизель
4038     бензин
Name: fuelType, dtype: object

In [245]:
train_mod.fuelType.unique() 

array(['бензин', 'дизель', 'электро', 'гибрид', 'газ'], dtype=object)

In [246]:
test_mod.fuelType.unique()

array(['бензин', 'дизель', 'гибрид', 'электро', 'газ'], dtype=object)

Никаких преобразований не требуется

## mileage

In [247]:
column = 'mileage'
type_comparison(column)

TEST


15902    125000
4270     293413
23422    185117
26046     18385
29913    204651
Name: mileage, dtype: int64

TRAIN


29667    115000
80286         0
31895     93628
2880          0
22199    105000
Name: mileage, dtype: int64

Преобразований не требуется

## modelDate

In [248]:
column = 'modelDate'
type_comparison(column)

TEST


33801    2011
29936    2015
31783    2004
13038    2013
33955    1991
Name: modelDate, dtype: int64

TRAIN


69647    2015.0
64670    2006.0
41909    2018.0
87436    2005.0
59620    2018.0
Name: modelDate, dtype: float64

Приводим к единому формату

In [249]:
#Преобразование
train_mod.modelDate = train_mod.modelDate.convert_dtypes(int) #astype('int')

In [250]:
column = 'modelDate'
type_comparison(column)

TEST


30923    1988
12856    2011
13837    2002
19538    2006
32828    2010
Name: modelDate, dtype: int64

TRAIN


76202    2010
78490    2019
27779    2005
47434    2002
30489    2001
Name: modelDate, dtype: Int64

## name

In [251]:
column = 'name'
type_comparison(column)

TEST


18521    320 3.2 AT (218 л.с.) 4WD
2480            1.2 AMT (105 л.с.)
6357         2.5 AT (210 л.с.) 4WD
20197            3.5 AT (277 л.с.)
24496       2.5d AT (174 л.с.) 4WD
Name: name, dtype: object

TRAIN


22461                     2.5 AT (150 л.с.)
82105    350 BlueTEC 3.0d AT (249 л.с.) 4WD
61440          Scout 2.0d MT (140 л.с.) 4WD
13431                     1.6 MT (109 л.с.)
63636                2.0 CVT (268 л.с.) 4WD
Name: name, dtype: object

В этом столбце есть информация об объёме двигателя. Её можно использовать для заполнения пропусков в столбце 'engineDisplacement'.  
Потом этот столбец будет не нужен.

In [252]:
columns_to_drop.append('name')

In [253]:
#Заменим все значения в 'engineDisplacement'
train_mod['engineDisplacement'] = train_mod['name'].apply(pull_EngineDisplacement)

In [254]:
#Проверим результат
train_mod[train_mod['engineDisplacement'] == 0]['fuelType'].unique()

array(['электро'], dtype=object)

In [255]:
train_mod[train_mod['engineDisplacement'].isna()]

,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,...,Состояние,Владельцы,ПТС,Таможня,Владение,price,start_date,hidden,model,Country_of_origin


Пропусков нет, а у электромобилей объём двигателя равен 0

## numberOfDoors

In [256]:
column = 'numberOfDoors'
type_comparison(column)

TEST


1024     5
33222    5
26408    3
26829    5
12938    4
Name: numberOfDoors, dtype: int64

TRAIN


27329    5.0
29746    4.0
11353    5.0
33429    5.0
8784     5.0
Name: numberOfDoors, dtype: float64

Приводим к единому формату

In [257]:
#Преобразование
train_mod.numberOfDoors = train_mod.numberOfDoors.convert_dtypes(int)

## productionDate

In [258]:
column = 'productionDate'
type_comparison(column)

TEST


5874     2008
14869    2008
5693     2008
9621     2010
12118    2017
Name: productionDate, dtype: int64

TRAIN


51184    2020
3555     2020
9106     2018
8169     1998
77403    2006
Name: productionDate, dtype: int64

Не требует преобразований

## vehicleConfiguration

In [259]:
column = 'vehicleConfiguration'
type_comparison(column)

TEST


13426      ALLROAD_5_DOORS VARIATOR 2.0
20443    ALLROAD_5_DOORS MECHANICAL 4.2
15161               SEDAN AUTOMATIC 2.0
21655      ALLROAD_5_DOORS VARIATOR 2.0
4887      ALLROAD_5_DOORS AUTOMATIC 3.0
Name: vehicleConfiguration, dtype: object

TRAIN


55624     HATCHBACK_5_DOORS AUTOMATIC 1.6
69161                 COUPE AUTOMATIC 3.0
47953    HATCHBACK_5_DOORS MECHANICAL 1.6
37320                SEDAN MECHANICAL 1.6
11741       ALLROAD_5_DOORS AUTOMATIC 3.0
Name: vehicleConfiguration, dtype: object

Этот столбец тоже можно использовать для заполнения пропусков в столбце 'engineDisplacement'.  
После чего он также должен быть удалён.

In [260]:
columns_to_drop.append('vehicleConfiguration')

## vehicleTransmission

In [261]:
column = 'vehicleTransmission'
type_comparison(column)

TEST


24801        механическая
18634      автоматическая
16688      автоматическая
23249      автоматическая
30642    роботизированная
Name: vehicleTransmission, dtype: object

TRAIN


62121    MECHANICAL
50332    MECHANICAL
49098      VARIATOR
79526     AUTOMATIC
48979      VARIATOR
Name: vehicleTransmission, dtype: object

К единому формату

In [262]:
#Преобразование
train_mod.vehicleTransmission.unique()

array(['MECHANICAL', 'AUTOMATIC', 'ROBOT', 'VARIATOR'], dtype=object)

In [263]:
test_mod.vehicleTransmission.unique()

array(['роботизированная', 'механическая', 'автоматическая', 'вариатор'],
      dtype=object)

In [264]:
test_mod = test_mod.replace({'vehicleTransmission':
                            {'автоматическая': 'AUTOMATIC',
                            'вариатор': 'VARIATOR',
                            'механическая': 'MECHANICAL',
                            'роботизированная': 'ROBOT'}})

## Владельцы

# !!!Вот эту часть нужно как-то оформить красиво и доступно!!!

In [265]:
column = 'Владельцы'
type_comparison(column)

TEST


25885    3 или более
32089    3 или более
19012    2 владельца
18666    3 или более
3017     2 владельца
Name: Владельцы, dtype: object

TRAIN


16480    3.0
8347     2.0
82726    2.0
85774    NaN
17546    3.0
Name: Владельцы, dtype: float64

In [266]:
test_mod.Владельцы.unique()

array(['3 или более', '1\xa0владелец', '2\xa0владельца'], dtype=object)

In [267]:
train_mod.Владельцы.unique()

array([ 3.,  1.,  2., nan])

Приведём к единому формату

In [268]:
#Преобразование
train_mod.Владельцы = train_mod.Владельцы.convert_dtypes(int)

test_mod = test_mod.replace({'Владельцы':
                            {'1\xa0владелец': 1,
                            '2\xa0владельца': 2,
                            '3 или более': 3}})

Посчитаем % пропусков

In [269]:
col = 'Владельцы'

pct_missing_1 = np.mean(test_mod[col].isnull()) 
print('TEST')
print('{} - {}%'.format(col, round(pct_missing_1*100)))
pct_missing_2 = np.mean(train_mod[col].isnull()) 
print('TRAIN')
print('{} - {}%'.format(col, round(pct_missing_2*100)))

TEST
Владельцы - 0%
TRAIN
Владельцы - 14%


Видим пропуски в TRAIN. Возможно, их удастся как-нибудь заполнить.  
Кол-во владельцев может быть связано с пробегом и годом выпуска автомобиля. Проверим

In [270]:
#train_mod[train_mod['Владельцы'].isna()]['mileage'].unique()

In [271]:
#train_mod[train_mod['Владельцы'].isna()]['productionDate'].unique()

Пропуски есть только у автомобилей с нулевым пробегом, однако, судя по годам выпуска, не все они являлись новыми.

In [272]:
#train_mod[train_mod['Владельцы'].isna()].query('productionDate == 2017')

In [273]:
#train_mod[train_mod['Владельцы'].isna()].query('productionDate == 2018')

In [274]:
#train_mod[train_mod['Владельцы'].isna()].query('productionDate == 2019')

In [275]:
#train_mod[train_mod['Владельцы'].isna()].query('productionDate == 2020')

Все эти автомобили - леворульные. Очевидно, они были привезены из Японии. Отсутствие пробега, вероятно, значит отсутствие пробега по России. Это говорит о том, что и владельцев в России у этих автомобилей не было. Следует заполнить пропуски нолями

In [276]:
train_mod['Владельцы'].fillna(0, inplace = True)

## Владение

In [277]:
column = 'Владение' # перевести в дни, можно выкинуть
type_comparison(column)

TEST


31110                   NaN
23640      9 лет и 4 месяца
22228    6 лет и 10 месяцев
32866    2 года и 9 месяцев
25614                10 лет
Name: Владение, dtype: object

TRAIN


58284    {'year': 2018, 'month': 2}
267                             NaN
74060                           NaN
13314                           NaN
49031                           NaN
Name: Владение, dtype: object

Посчитаем % пропусков

In [278]:
col = 'Владение'

pct_missing_1 = np.mean(test_mod[col].isnull()) 
print('TEST')
print('{} - {}%'.format(col, round(pct_missing_1*100)))
pct_missing_2 = np.mean(train_mod[col].isnull()) 
print('TRAIN')
print('{} - {}%'.format(col, round(pct_missing_2*100)))

TEST
Владение - 65%
TRAIN
Владение - 67%


Большой процент пропусков в TRAIN и TEST (около 70%), столбец можно удалить.

In [279]:
columns_to_drop.append('Владение')

## Привод

In [280]:
column = 'Привод'
type_comparison(column)

TEST


6605       полный
33917    передний
11820    передний
9773       полный
2510     передний
Name: Привод, dtype: object

TRAIN


83243      задний
3669       полный
20753    передний
956      передний
78939      полный
Name: Привод, dtype: object

Оставляем как есть

## ПТС

In [281]:
column = 'ПТС'
type_comparison(column)

TEST


26788    Оригинал
33463    Оригинал
30152    Оригинал
1919     Оригинал
13943    Оригинал
Name: ПТС, dtype: object

TRAIN


2965     ORIGINAL
32992    ORIGINAL
56870    ORIGINAL
87856    ORIGINAL
14812    ORIGINAL
Name: ПТС, dtype: object

К единому формату

In [282]:
#Преобразование
test_mod = test_mod.replace({'ПТС':
                            {'Оригинал': 'ORIGINAL',
                            'Дубликат': 'DUPLICATE'}})

## Руль

In [283]:
column = 'Руль'
type_comparison(column)

TEST


34107    Левый
20207    Левый
24746    Левый
27017    Левый
15381    Левый
Name: Руль, dtype: object

TRAIN


26815    LEFT
35222    LEFT
55978    LEFT
69662    LEFT
54705    LEFT
Name: Руль, dtype: object

Преобразуем также как предыдущий столбец

In [284]:
#Преобразование
test_mod = test_mod.replace({'Руль':
                            {'Левый': 'LEFT',
                            'Правый': 'RIGHT'}})

## Состояние

In [285]:
column = 'Состояние'
type_comparison(column)

TEST


7540     Не требует ремонта
4620     Не требует ремонта
23029    Не требует ремонта
17598    Не требует ремонта
1269     Не требует ремонта
Name: Состояние, dtype: object

TRAIN


62859   NaN
39862   NaN
57091   NaN
66362   NaN
23106   NaN
Name: Состояние, dtype: float64

In [286]:
test_mod.Состояние.unique()

array(['Не требует ремонта'], dtype=object)

In [287]:
train_mod.Состояние.unique()

array([nan])

В TEST только одно значение, в TRAIN - пустой столбец. СОСТОЯНИЕ можно удалить.

In [288]:
columns_to_drop.append('Состояние')

## Таможня

In [289]:
column = 'Таможня'
type_comparison(column)

TEST


13736    Растаможен
26692    Растаможен
9272     Растаможен
987      Растаможен
12915    Растаможен
Name: Таможня, dtype: object

TRAIN


43096    True
22082    True
2842     True
54726    True
84028    True
Name: Таможня, dtype: bool

In [290]:
test_mod.Таможня.unique()

array(['Растаможен'], dtype=object)

In [291]:
train_mod.Таможня.unique()

array([ True])

В TRAIN и TEST во всех строках одинаковое значение, столбец ТАМОЖНЯ можно удалить.

In [292]:
columns_to_drop.append('Таможня')

In [293]:
#Удаляем лишние общие столбцы
test_mod.drop(columns = columns_to_drop, inplace = True)
train_mod.drop(columns = columns_to_drop, inplace = True)

In [294]:
test_mod.columns

Index(['bodyType', 'brand', 'car_url', 'color', 'complectation_dict',
       'description', 'engineDisplacement', 'enginePower', 'equipment_dict',
       'fuelType', 'image', 'mileage', 'modelDate', 'model_info', 'model_name',
       'numberOfDoors', 'parsing_unixtime', 'priceCurrency', 'productionDate',
       'sell_id', 'super_gen', 'vehicleTransmission', 'vendor', 'Владельцы',
       'ПТС', 'Привод', 'Руль', 'Country_of_origin'],
      dtype='object')

In [295]:
train_mod.columns

Index(['bodyType', 'brand', 'color', 'fuelType', 'modelDate', 'numberOfDoors',
       'productionDate', 'vehicleTransmission', 'engineDisplacement',
       'enginePower', 'description', 'mileage', 'Комплектация', 'Привод',
       'Руль', 'Владельцы', 'ПТС', 'price', 'start_date', 'hidden', 'model',
       'Country_of_origin'],
      dtype='object')

In [296]:
#Удалим лишние столбцы из Теста чтобы датасеты соответствовали друг другу 
test_drop = []
for i in range(len(test_mod.columns)):
    if test_mod.columns[i] not in train_mod.columns:
        test_drop.append(test_mod.columns[i])

In [297]:
test_mod.drop(columns = test_drop, inplace = True)

In [298]:
train_drop = []
for i in range(len(train_mod.columns)):
    if train_mod.columns[i] not in test_mod.columns:
        train_drop.append(train_mod.columns[i])

In [299]:
train_mod.drop(columns = train_drop, inplace = True)

In [300]:
test_mod.columns.sort_values() == train_mod.columns.sort_values()

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True])

In [301]:
#Добавим к обоим датасетам столбец 'TEST' и заполним единицами в test_mod и нолями в train_mod
#Так их легко можно будет разделить позже
test_mod['TEST'] = 1
train_mod['TEST'] = 0

In [302]:
#Объединяем
whole_data = train_mod.append(test_mod)

C:\Temp\ipykernel_2416\3064683694.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  whole_data = train_mod.append(test_mod)


In [303]:
whole_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 121949 entries, 0 to 34685
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   bodyType             121949 non-null  object 
 1   brand                121949 non-null  object 
 2   color                121949 non-null  object 
 3   fuelType             121949 non-null  object 
 4   modelDate            121949 non-null  Int64  
 5   numberOfDoors        121949 non-null  Int64  
 6   productionDate       121949 non-null  int64  
 7   vehicleTransmission  121949 non-null  object 
 8   engineDisplacement   121949 non-null  float64
 9   enginePower          121949 non-null  Int64  
 10  description          118716 non-null  object 
 11  mileage              121949 non-null  int64  
 12  Привод               121949 non-null  object 
 13  Руль                 121949 non-null  object 
 14  Владельцы            121949 non-null  Int64  
 15  ПТС               